# GenericSelectors

GenericSelectors let you do any kind of modifications to the building that you want from a custom function. This custom function gets the building and a value as input.

For example, you can do calculations before setting some fields or read values from the building.
You can change as many fields as you like, or even add/delete building objects.

GenericSelectors also support custom `get` functions to read the values they apply to and `setup` functions which run before values are set.

In [ ]:
from besos import eppy_funcs as ef, sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import (
    FieldSelector,
    GenericSelector,
    Parameter,
    RangeParameter,
)
from besos.problem import EPProblem

from errors import ModeError

In [ ]:
# Load Building
json_building = ef.get_building(mode="json")

If we want to modify a building directly, we can do it like this

In [ ]:
print("before:", json_building["Lights"]["Core_ZN_Lights"]["fraction_radiant"])
json_building["Lights"]["Core_ZN_Lights"]["fraction_radiant"] = 0.6
print("after:", json_building["Lights"]["Core_ZN_Lights"]["fraction_radiant"])

We can write a funtion which sets this field's value on any building and value that it is given:

In [ ]:
def set_core_lights_fraction_radiant(json_building, value):
    json_building["Lights"]["Core_ZN_Lights"]["fraction_radiant"] = value

If we put this function in a selector, it can perform this modification based on values input to an evaluator.
The following two selectors transform a building in the same way.

In [ ]:
selector1 = GenericSelector(set=set_core_lights_fraction_radiant)

selector2 = FieldSelector(
    class_name="Lights", object_name="Core_ZN_Lights", field_name="fraction_radiant"
)

In [ ]:
# they are equivalent when setting values on a building
b1 = ef.get_building(mode="json")
selector1.set(b1, 0.65)

b2 = ef.get_building(mode="json")
selector2.set(b2, 0.65)

print(b1 == b2)

In [ ]:
# they are also equivalent when used in evaluators
descriptor = RangeParameter(0.5, 0.8)
evaluator1 = EvaluatorEP(EPProblem([Parameter(selector1, descriptor)]), json_building)
evaluator2 = EvaluatorEP(EPProblem([Parameter(selector2, descriptor)]), json_building)
print(evaluator1([0.65]), evaluator2([0.65]))

## Supporting idf and json
As written, the function `set_core_lights_fraction_radiant` cannot handle idfs.

In [ ]:
idf_building = ef.get_building(mode="idf")
try:
    set_core_lights_fraction_radiant(idf_building, 0.5)
except TypeError as e:
    print(e)

We can re-write our function to use eppy (documented [here](https://pythonhosted.org/eppy/Main_Tutorial.html)) to manipulate the idf.

In [ ]:
def set_core_lights_fraction_radiant_idf(idf_building, value):
    idf_building.idfobjects["LIGHTS"][0].Fraction_Radiant = value


set_core_lights_fraction_radiant_idf(idf_building, 0.6)

We can also write a function which works on both json and idfs

In [ ]:
def set_core_lights_fraction_radiant_anyfile(building, value):
    mode = ef.get_mode(building)
    if mode == "json":
        building["Lights"]["Core_ZN_Lights"]["fraction_radiant"] = value
    elif mode == "idf":
        building.idfobjects["LIGHTS"][0].Fraction_Radiant = value
    else:
        # this should never happen, unless a new file type is introduced.
        # including this else clause is optional
        raise ModeError(mode)


set_core_lights_fraction_radiant_anyfile(idf_building, 0.6)
set_core_lights_fraction_radiant_anyfile(json_building, 0.6)

## Using Selectors in custom functions
Using a selector inside our custom function can often make them easier to write

In [ ]:
core_lights = FieldSelector(
    class_name="Lights", object_name="Core_ZN_Lights", field_name="fraction_radiant"
)


def set_core_lights_fraction_radiant_simple(building, value):
    core_lights.set(building, value)


# Since FieldSelectors support json and idfs, our code works with both automatically
set_core_lights_fraction_radiant_anyfile(idf_building, 0.6)
set_core_lights_fraction_radiant_anyfile(json_building, 0.6)

## Where this starts being useful
So far, the examples have just been setting a field, which `FieldSelector`s do perfectly well.
However, sometimes we want to do something more complex than set a field.

In [ ]:
lights_selector = FieldSelector(
    class_name="Lights", object_name="*", field_name="Watts per Zone Floor Area",
)

This function sets the equipment `Watts per Zone Floor Area` to the value it is given, and then sets the wattage for the lights to `25 - value`. This makes sure that the wattage used by the lights and equipment together stays constant. Also, since our selectors apply to all Lights/ElectricEquipment objects, they are all affected by the function, not just the ones printed below.

Selectors can be defined both inside a generic selector function or before the function call. Selector defined outside a function can be reused elsewhere.

In [ ]:
def set_equipment_and_lights(building, value):
    # dummy variable
    total = 25
    # Create a selector to change.
    equipment_selector = FieldSelector(
        class_name="ElectricEquipment",
        object_name="*",
        field_name="Watts per Zone Floor Area",
    )
    # change the fields of the building
    equipment_selector.set(building, value)
    lights_selector.set(building, total - value)


set_equipment_and_lights(
    json_building, 2
)  # idfs are also supported automatically, since we are using built-in selectors

print(
    json_building["ElectricEquipment"]["Core_ZN_MiscPlug_Equip"][
        "watts_per_zone_floor_area"
    ],
    json_building["Lights"]["Core_ZN_Lights"]["watts_per_zone_floor_area"],
)

## A more advanced example
We can get a list of **objects** affected by a selector using the `.get_objects` method. This lets us iterate over all the objects a selector applies to.

In [ ]:
# reload the building
idf_building = ef.get_building(mode="idf")
json_building = ef.get_building(mode="json")

In [ ]:
# this reuses the lights_selector defined in the last example.
lights_selector.get_objects(json_building)

To make the following example more interesting, let's change the `Watts per Zone Floor Area` of one of the lights. (`[0]` gets the first entry from the list)

In [ ]:
lights_selector.get_objects(json_building)[0]["watts_per_zone_floor_area"] = 5
lights_selector.get_objects(idf_building)[0].Watts_per_Zone_Floor_Area = 5

The following function takes the current value of a light's wattage, and multiplies it by the value `multiplier`

In [ ]:
def multiply_lights(building, multiplier):
    """multiplies the lights wattage field by the multiplier argument"""
    mode = ef.get_mode(building)
    objects = lights_selector.get_objects(building)
    if mode == "idf":
        for o in objects:
            current_value = o.Watts_per_Zone_Floor_Area
            new_value = multiplier * current_value
            o.Watts_per_Zone_Floor_Area = new_value
    elif mode == "json":
        for o in objects:
            current_value = o["watts_per_zone_floor_area"]
            new_value = multiplier * current_value
            o["watts_per_zone_floor_area"] = new_value
    else:
        raise ModeError(mode)

In [ ]:
lights_multiplier = GenericSelector(set=multiply_lights)

In [ ]:
# this doubles the wattage of all lights. each time it is rerun, it will double them again.
lights_multiplier.set(idf_building, 2)
idf_building.idfobjects["LIGHTS"][:2]  # remove [:2] to see all of the objects

In [ ]:
# We can run simulations with a GenericParameter

# note that the lights selector is only used to help build the GenericSelector
# using it normally (inside a parameter) would replace the values in the idf

# reload the building to clear any changes
building = ef.get_building()  # both modes work here

parameters = [
    Parameter(
        selector=GenericSelector(set=multiply_lights),
        value_descriptor=RangeParameter(0.5, 2),
        name="multiplier",
    )
]
problem = EPProblem(inputs=parameters)
evaluator = EvaluatorEP(problem, building)
samples = sampling.dist_sampler(sampling.lhs, problem, num_samples=5)

In [ ]:
outputs = evaluator.df_apply(samples, keep_input=True)
outputs.sort_values(by="Electricity:Facility")
# unsurprisingly, increasing the lighting wattage increases total electricity use

## `setup` and `get` functions
GenericSelectors also support `get` and `setup` functions.

Setup functions do something to the building before all of the modifications of other functions have been applied. They will run once when an evaluator is created, modifying the building for all runs of that evaluator.

get functions provide an option to have the selector read values for you. They should return a list of values, one for each of the building-objects the selector applies to. (Technically you can return any value you want from your get function, since you define it.)
Get function are not used internally at the time of writing, and may be changed to make them more useful.

In [ ]:
def delete_one_fenstration_surface(building):
    mode = ef.get_mode(building)
    if mode == "idf":
        building.idfobjects["FENESTRATIONSURFACE:DETAILED"].pop(0)
    elif mode == "json":
        building["FenestrationSurface:Detailed"].pop(
            "Perimeter_ZN_1_wall_south_Window_1"
        )
    else:
        raise ModeError(mode)


glazing_selector = FieldSelector(
    class_name="WindowMaterial:SimpleGlazingSystem",
    field_name="Solar Heat Gain Coefficient",
)


def set_glazing_coefficient(building, value):
    glazing_selector.set(building, value)


def get_glazing_coefficient(building):
    return glazing_selector.get(building)

In [ ]:
idf_building, json_building = ef.get_building(mode="idf"), ef.get_building(mode="json")
print(
    "before setup",
    len(idf_building.idfobjects["FENESTRATIONSURFACE:DETAILED"]),
    len(json_building["FenestrationSurface:Detailed"]),
)
delete_one_fenstration_surface(json_building)
delete_one_fenstration_surface(idf_building)
print(
    "after setup ",
    len(idf_building.idfobjects["FENESTRATIONSURFACE:DETAILED"]),
    len(json_building["FenestrationSurface:Detailed"]),
)

custom setter, custom getter, and custom setup.

In [ ]:
selector = GenericSelector(
    set=set_glazing_coefficient,
    get=get_glazing_coefficient,
    setup=delete_one_fenstration_surface,
)

print("before set", selector.get(idf_building))
selector.set(idf_building, 0.42)
print("after set ", selector.get(idf_building))